# Chapter 9 - 파이썬스러운 객체

## 객체 표현

- repr(): 객체를 **개발자**가 보고자 하는 형태로 표현한 문자열로 반환한다.
- str(): 객체를 **사용자**가 보고자 하는 형태로 표현한 문자열로 반환한다.

repr()와 str() 메서드를 지원하려면 \_\_repr\_\_()와 \_\_str\_\_() 특별 메서드를 구현할 것.

## 벡터 클래스

In [4]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))    

    def __format__(self, fmt_spec=''):
        components = (format(c, fmt_spec) for c in self)
        return '({}, {})'.format(*components)

In [6]:
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

In [7]:
format(v1, '.3f')

'(3.000, 4.000)'

In [8]:
hash(v1)

TypeError: unhashable type: 'Vector2d'

In [2]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

In [3]:
v1 = Vector2d(3, 4)
v2 = Vector2d(4, 3)

print(hash(v1))
print(hash(v2))  # 같게 나오면 안 되지 않나?

7
7


## 파이썬에서의 비공개 속성과 보호된 속성

파이썬에는 C++이나 자바의 private처럼 비공개 멤버변수를 생성할 수 없지만<br>
'비공개' 성격의 속성을 실수로 변경하지 못하게 하는 간단한 메커니즘은 있다.<br>
<br>
속성명을 \_\_mood처럼 두 개의 언더바로 시작하고 언더바 없이 혹은 하나의 언더바로 끝나도록 정의하면<br>
파이썬은 언더바와 클래스명을 변수명 앞에 붙여 객체의 \_\_dict\_\_에 저장한다.

In [4]:
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [5]:
v1._Vector2d__x

3.0

속성명 앞에 언더바 하나를 붙이더라도 파이썬 인터프리터가 특별히 처리하는 것은 없지만<br>
클래스 외부에서 그런 속성에 접근하지 않는 것은 파이썬 프로그래머 사이에 일종의 금기처럼 자리잡혀 있다.<br>
파이썬 문서 일부에선 단일 언더바로 시작하는 속성을 '보호된' 속성이라고 부르기도 한다.

## \_\_slots\_\_ 클래스 속성으로 공간 절약하기

기본적으로 파이썬은 객체 속성을 각 객체 안의 \_\_dict\_\_라는 딕셔너리형 속성에 저장하므로 메모리 사용량 부담이 상당히 크다.<br>
\_\_slots\_\_ 속성은 파이썬 인터프리터가 객체 속성을 딕셔너리 대신 튜플에 저장하게 만든다.<br>
사용법은 다음과 같다.

```python
class Vector2d:
    __slots__ = ('__x', '__y')
    
    # ...
```

위키백과의 창시자이자 익스트림 프로그래밍의 선구자인 워드 커닝햄은 '작동하는 가장 단순한 방법은 무엇인가?'라는 질문을 하라고 권장한다.<br>
**목적에 집중하라**는 의미이다.<br>
시작부터 게터와 세터를 구현하는 것은 목적을 혼란스럽게 만든다.<br>
파이썬에서는 나중에 필요해지면 프로퍼티를 바꿀 수 있다고 생각하면서 간단히 공개 속성으로 시작할 수 있다.